In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import pickle

DIRECTORY = 'data'

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

In [ ]:
def load_data():
    def unpickle(file):
        with open(file, 'rb') as fo:
            dict = pickle.load(fo, encoding='bytes')
        return dict
    names = [n.decode('utf-8')
             for n in unpickle(DIRECTORY + "/batches.meta")[b'label_names']]
    x_train = None
    y_train = []

    for i in range(1, 2):
        data = unpickle(DIRECTORY + "/data_batch_" + str(i))
        if i > 1:
            x_train = np.append(x_train, data[b'data'], axis=0)
        else:
            x_train = data[b'data']
        y_train += data[b'labels']

    data = unpickle(DIRECTORY + "/test_batch")

    x_test = data[b'data']
    y_test = data[b'labels']

    return names, x_train, y_train, x_test, y_test


In [ ]:
def plot_tensor(tensor, perm=None):
    if perm == None:
        perm = (1, 2, 0)
    plt.figure()
    plt.imshow(tensor.permute(perm).cpu().numpy().astype(np.uint8))
    plt.show()


In [ ]:
names, x_train, y_train, x_test, y_test = load_data()
names

In [ ]:
x_train_tensors = torch.tensor(x_train).to(device)
y_train_tensors = torch.tensor(y_train).to(device)
x_train_tensors = torch.reshape(x_train_tensors, [10000, 3, 32, 32])
x_test_tensors = torch.tensor(x_train).to(device)
y_test_tensors = torch.tensor(y_train).to(device)
x_test_tensors = torch.reshape(x_test_tensors, [10000, 3, 32, 32])

x_test_tensors.shape


In [ ]:
from torch.nn import Module
from torch.nn import Conv2d
from torch.nn import Linear
from torch.nn import MaxPool2d
from torch.nn import Sigmoid
from torch.nn import LogSoftmax
from torch import flatten

class Net(Module):
  def __init__(self, num_channels, classes) -> None:
    super(Net, self).__init__()
    
    # first set | conv_layer -> relu -> pool
    self.conv1 = Conv2d(in_channels=num_channels, out_channels=6, kernel_size=(3, 3), padding='same')
    self.sig1 = Sigmoid()
    self.maxpool1 = MaxPool2d(kernel_size=(2, 2), stride=(2, 2))

    # second set | conv -> relu -> pool
    self.conv2 = Conv2d(in_channels=6, out_channels=9, kernel_size=(5, 5), padding='same')
    self.sig2 = Sigmoid()
    self.maxpool2 = MaxPool2d(kernel_size=(2, 2), stride=(2, 2))

    # initializa fully connected layer
    self.fc1 = Linear(in_features=576, out_features=1000)
    self.sig3 = Sigmoid()

    # initialize softmax classifier
    self.fc2 = Linear(in_features=1000, out_features=classes)
    self.log_soft_max = LogSoftmax(dim=1)

  def forward(self, x):
    # the x data passes through our first set 
    # conv -> relu -> pool layers
    x = self.conv1(x)
    x = self.sig1(x)
    x = self.maxpool1(x)

    # passes the output from previous layer to through the second

    x = self.conv2(x)
    x = self.sig2(x)
    x = self.maxpool2(x)

    # flatten the output and passes it
    x = flatten(x, 1)
    x = self.fc1(x)
    x = self.sig3(x)

    # pass the output to our softmax to get our final output
    x = self.fc2(x)
    return self.log_soft_max(x)

In [ ]:
def get_max(output):
  return torch.argmax(output, dim=1)

In [ ]:
import torch.optim as optim
from torch import nn

net = Net(num_channels=3, classes=10)
net.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=1e-5, momentum=0.9999)

In [ ]:

loss_val = 0.0

for e in range(150):
  net.train()

  pred = net(x_train_tensors.float())

  # total_train_loss = 0
  # total_val_loss = 0

  # train_correct = 0
  # val_correct = 0

  loss = criterion(pred, y_train_tensors)
  loss.backward()

  optimizer.step()

get_max(pred)

In [ ]:
y_train_tensors

In [ ]:
plot_tensor(x_train_tensors[1])